# Veremos como resolver el problema de bases de datos no balanceadas

In [1]:
# Cargamos las librerias

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, RepeatedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics

# Cargar los datos

datos=pd.read_csv("https://raw.githubusercontent.com/Albertuff/Machine-Learning/master/datos/default.csv")

# Definimos los atributos(features) y las variable objeivo(target)

X=datos[["Empleado","Balance","Salario_anual"]]           # Atributos
Y=datos["Impago"]                                         # Respuesta

### Cómo notamos que hay desbalanceo en los datos?

In [2]:
pd.value_counts(Y)

0    9667
1     333
Name: Impago, dtype: int64

### Tengo mas clientes puntuales (9667) que impuntuales (333)

### Clasificación cruda de la base de datos

In [3]:
from sklearn.metrics import classification_report

# Dividimos el conjunto de datos en prueba y entrenamiento

x_train, x_test, y_train, y_test=train_test_split(X,Y,test_size=0.2,random_state=1234,stratify=Y) 
#stratify =Y garantiza que todo el conjunto de prueba y entrenamiento tenfan 

# Definimos el modelo y lo entrenamos

modelo=KNeighborsClassifier(n_neighbors=4,n_jobs=-1).fit(x_train,y_train)

# Hacemos las predicciones

predicciones=modelo.predict(x_test)
print(classification_report(y_test,predicciones))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1934
           1       0.50      0.06      0.11        66

    accuracy                           0.97      2000
   macro avg       0.73      0.53      0.55      2000
weighted avg       0.95      0.97      0.95      2000



# Resampling: Sobremuestreo y Submuestreo
## Sobremuestreo
Se sobremuestrea la clase minoritaria para tener tantos 1's como 0's

In [4]:
from sklearn.utils import resample

train, test=train_test_split(datos,test_size=0.2,random_state=1234)

# Vamos a definir la clase mayoritaria
clase_mayoritaria=train[Y==0]
# Definimos la clase minoritaria
clase_minoritaria=train[Y==1]

m=len(clase_mayoritaria)    # Elementos en la clase mayoritaria 7733

# Generamos una muestra de 7733 casos de clientes que caen en impago
sobremuestra=resample(clase_minoritaria,replace=True,n_samples=m,random_state=1234)

# El nuevo conjunto de entrenamiento es:
muestra_nueva=pd.concat([clase_mayoritaria,sobremuestra])
print(muestra_nueva)
y=muestra_nueva.Impago
pd.value_counts(y)


      Empleado   Balance  Salario_anual  Impago
5525         1   9020.28      615927.84       0
5034         0   8238.36      309357.36       0
4218         1  17717.76      608422.20       0
917          1   9839.04      347521.32       0
6637         1  11665.08      314773.80       0
...        ...       ...            ...     ...
932          1  14922.60      451612.20       1
1939         0  20984.16      164585.04       1
3156         1  22794.60      552914.88       1
4952         1  12345.24      484161.96       1
9522         1  11509.92      713222.76       1

[15466 rows x 4 columns]


C:\Users\alber\AppData\Local\Temp\ipykernel_9588\1567651043.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clase_mayoritaria=train[Y==0]
C:\Users\alber\AppData\Local\Temp\ipykernel_9588\1567651043.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clase_minoritaria=train[Y==1]


0    7733
1    7733
Name: Impago, dtype: int64

In [5]:
# Ahora muestra_nueva es la nueva base de datos en la cual tenemos tantos 1's como 0's 


X=muestra_nueva[["Empleado","Balance","Salario_anual"]]           # Atributos
Y=muestra_nueva["Impago"]                                         # Respuesta

x_train, x_test, y_train, y_test=train_test_split(X,Y,test_size=0.2,random_state=1234)

# Definimos el modelo y lo entrenamos

modelo=KNeighborsClassifier(n_neighbors=4,n_jobs=-1).fit(x_train,y_train)

# Hacemos las predicciones

predicciones=modelo.predict(x_test)
print(classification_report(y_test,predicciones))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1533
           1       0.96      1.00      0.98      1561

    accuracy                           0.98      3094
   macro avg       0.98      0.98      0.98      3094
weighted avg       0.98      0.98      0.98      3094



# Submuestreo

In [6]:
from sklearn.utils import resample
# Se submuestrea la clase mayoritaria para tener tantos 0's como 1's. (Voy a tomar menos elementos de la clase mayoritaria)

train, test=train_test_split(datos,test_size=0.2,random_state=1234)

# Vamos a definir la clase mayoritaria
clase_mayoritaria=train[train.Impago==0]
# Definimos la clase minoritaria
clase_menor=train[train.Impago==1]

m=len(clase_menor)    # Elementos en la clase menor

# Generamos una muestra de 267 casos de clientes que pagan puntualmente
submuestra=resample(clase_mayoritaria,replace=False,n_samples=m,random_state=1234)

# El nuevo conjunto de entrenamiento es:
muestra_nueva=pd.concat([clase_menor,submuestra])
print(muestra_nueva)
y=muestra_nueva.Impago
pd.value_counts(y)

      Empleado   Balance  Salario_anual  Impago
4709         1  24902.28      490589.88       1
3921         1  19594.68      531918.96       1
8456         1  21789.48      379590.96       1
6847         0  23485.44      225671.40       1
7199         1  22003.80      522477.36       1
...        ...       ...            ...     ...
9974         0  11199.96      312616.80       0
1799         1   1122.84      515169.36       0
8559         0   7013.88      168208.68       0
9886         0  11076.24      229639.32       0
4701         1      0.00      399265.80       0

[534 rows x 4 columns]


1    267
0    267
Name: Impago, dtype: int64

In [7]:
# Ahora muestra_nueva es la nueva base de datos en la cual tenemos tantos 1's como 0's 


X=muestra_nueva[["Empleado","Balance","Salario_anual"]]           # Atributos
Y=muestra_nueva["Impago"]                                         # Respuesta

x_train, x_test, y_train, y_test=train_test_split(X,Y,test_size=0.2,random_state=1234)

# Definimos el modelo y lo entrenamos

modelo=KNeighborsClassifier(n_neighbors=4,n_jobs=-1).fit(x_train,y_train)

# Hacemos las predicciones

predicciones=modelo.predict(x_test)
print(classification_report(y_test,predicciones))

              precision    recall  f1-score   support

           0       0.72      0.94      0.82        50
           1       0.93      0.68      0.79        57

    accuracy                           0.80       107
   macro avg       0.83      0.81      0.80       107
weighted avg       0.83      0.80      0.80       107



# Balanceo con la tecnica (SMOTE)
# Synthetic Minority Oversampling Technique
Solo funciona para atributos continuos.

In [8]:
from imblearn.over_sampling import SMOTE #pip install imbalanced-learn

In [9]:
# Llamamos a la funcion SMOTE
smote=SMOTE(sampling_strategy="minority",random_state=1234)
# Vamos a sobremuestrear la clase minoritaria 

In [10]:
# Eliminamos la variable categorica : Empleado
X=datos[["Balance","Salario_anual"]]
Y=datos["Impago"]

In [16]:
# Agregamos un parametro adicional: stratify, sirve para que las muestras de prueba y entrenamiento mantengan la misma distribucion de respuestas que están en Y (target)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1234, stratify=Y)

# Balanceo de los datos, empleando smote

x_train_smote, y_train_smote=smote.fit_resample(X_train,Y_train)    # Aqui ya tenemos la muestra balanceada y no todos los casos son replicas, muchos son artificiales

modelo.fit(x_train_smote,y_train_smote)
y_predict=modelo.predict(X_test)
print(classification_report(Y_test,y_predict))

              precision    recall  f1-score   support

           0       0.98      0.93      0.96      1933
           1       0.23      0.58      0.33        67

    accuracy                           0.92      2000
   macro avg       0.61      0.76      0.64      2000
weighted avg       0.96      0.92      0.94      2000



In [15]:
y_train_smote.value_counts()

0    7734
1    7734
Name: Impago, dtype: int64